In [22]:
# coding:utf-8
import io
import os
import sys
import numpy as np
import pandas as pd
import pygame as pg
from pygame.locals import *

def readStim(filename):
    stimuli = pd.read_csv(filename,sep=',',encoding='gbk')
    try:
        del stimuli['Unnamed: 0']
    except:
        pass
    return stimuli
path='D:\\Coding\\py\\py3\\experiment\\'
stimuli= readStim(path+'stimli.csv')

pos = [0,0,1,2] * (len(stimuli)//4//4)
random_pos = []
for i in range(4):
    random_pos += np.random.permutation(pos).tolist()
stimuli['Pos'] = random_pos

In [19]:
import ctypes
import time
Objdll = ctypes.windll.LoadLibrary("inpoutx64.dll")
t=time.time()
Objdll.Out32(0xD010,1)
while 1:
    if (time.time()-t)>1:
        Objdll.Out32(0xD010,1)
        break
print(time.time()-t)

1.00099992752


In [16]:
class PParallelInpOut32(object):
    """
    This class provides read/write access to the parallel port on a PC
    using inpout32 (for instance for Windows 7 64-bit)
    """
    def __init__(self, address=0xD010):


        from numpy import uint8
        from ctypes import windll

        if isinstance(address, basestring) and address.startswith('0x'): #convert u"0x0378" into 0x0378
            self.base = int(address, 16)
        else:
            self.base = address
        self.port = windll.inpout32

        BYTEMODEMASK = uint8(1 << 5 | 1 << 6 | 1 << 7)

        # Put the port into Byte Mode (ECP register)
        self.port.Out32( self.base + 0x402,
                    int((self.port.Inp32(self.base + 0x402) & ~BYTEMODEMASK) | (1 << 5)) )

        # Now to make sure the port is in output mode we need to make
        # sure that bit 5 of the control register is not set
        self.port.Out32( self.base + 2, int(self.port.Inp32(self.base + 2) & ~uint8( 1 << 5 )) )
        self.status = None

    def setData(self, data):
        self.port.Out32( self.base, data )

    def setPin(self, pinNumber, state):
        """Set a desired pin to be high(1) or low(0).

        Only pins 2-9 (incl) are normally used for data output::

            parallel.setPin(3, 1)#sets pin 3 high
            parallel.setPin(3, 0)#sets pin 3 low
        """
        # I can't see how to do this without reading and writing the data
        if state:
            self.port.Out32( self.base, self.port.Inp32( self.base ) | (2**(pinNumber-2)) )
        else:
            self.port.Out32( self.base, self.port.Inp32( self.base ) & (255 ^ 2**(pinNumber-2)) )

1439705206.722

In [5]:
import ctypes
Objdll = ctypes.windll.LoadLibrary("inpoutx64.dll")
%timeit Objdll.Out32(0xD010,1)
%timeit Objdll.Out32(0xD010,31)

The slowest run took 60.92 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 595 ns per loop
The slowest run took 8.15 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 593 ns per loop


In [23]:
len(stimuli[(stimuli.WholeAcc==1)&(stimuli.LeftAcc==1)&(stimuli.Pos==0)])

60